# 从one-hot到word2vec

参考资料：
1.one-hot向量与word2vec
https://blog.csdn.net/mawenqi0729/article/details/80698780
2.word2vec 中的数学原理详解（一）目录和前言 
https://blog.csdn.net/itplus/article/details/37969519


## one-hot向量

词向量是通过一个数字组成的向量来表示一个词。假设在一个语料集合中，一共有n个不同的词，则可用一个长度为n的向量表示，例如只有第i个词存在，则向量index=i处值为1外，向量其他位置的值都为0，这样就可以唯一地通过一个[0,0,1,...,0,0]形式的向量表示一个词。

one-hot向量比较简单也容易理解，但是有很多问题。比如当加入新词时，整个向量的长度会改变，并且存在维度过高难以计算的问题，以及向量的表示方法很难体现两个词之间的关系。

优点：简单易懂、稀疏存储
缺点：维度灾难、词汇鸿沟

eg:
已知三个feature，三个feature分别取值如下： feature1=[“male”, “female”] feature2=[“from Europe”, “from US”, “from Asia”] feature3=[“uses Firefox”, “uses Chrome”, “uses Safari”, “uses Internet Explorer”]

如果做普通数据处理，那么我们就按0,1,2,3进行编号就行了。例如feature1=[0，1],feature2=[0，1，2],feature3=[0，1，2，3]。 

如果某个样本为[“male”,“from Asia”, “uses Chrome”]，它就可以表示为[0，2，1]。

In [2]:
# sklearn实现one-hot encode
from sklearn import preprocessing

enc = preprocessing.OneHotEncoder()  # 创建对象
enc.fit([[0,0,3],[1,1,0],[0,2,1],[1,0,2]])   # 拟合
print(enc.n_values_) #每个特征有几种取值
print(enc.feature_indices_) #特征索引
array = enc.transform([[0,1,3]]).toarray()  # 转化
print(array)


[2 3 4]
[0 2 5 9]
[[1. 0. 0. 1. 0. 0. 0. 0. 1.]]


C:\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function n_values_ is deprecated; The ``n_values_`` attribute was deprecated in version 0.20 and will be removed 0.22.
  warnings.warn(msg, category=DeprecationWarning)
C:\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:77: DeprecationWarning: Function feature_indices_ is deprecated; The ``feature_indices_`` attribute was deprecated 

enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]]) 

fit中，所有的数组第一个元素取值分别为：0，1，0，1，最大为1，且为两种元素（0，1），说明用2个状态位来表示就可以了，且该维度的value值为2

所有的数组第二个元素取值分别为：0，1，2，0，最大为2，且为两种元素（0，1，2），说明用3个状态位来表示就可以了，且该维度的value值为3

所有的数组第三个元素取值分别为：3，0，1，2，最大为3，且为两种元素（0，1，2，3），说明用4个状态位来表示就可以了，且该维度的value值为4

所以整个的value值为（2，3，4），这也就解释了 enc.n_values_等于array([2,3,4])的原因。

而enc.feature_indices_则是特征索引，该例子中value值为（2，3，4），则特征索引从0开始，到2的位置为第一个，到2+3=5的位置为第二个，到2+3+4的位置为第三个，索引为array([0,2,5,9])

enc.transform([[0,1,3]]).toarray()
第一个特征(可为0、1)在该样本中取0，编码为[1,0]
第二个特征(可为0、1、2)在该样本中取1，编码为[0，1，0]
第三个特征(可为0、1、2，3)在该样本中取3，编码为[0，0，0，1]
连在一起就是[[1. 0. 0. 1. 0. 0. 0. 0. 1.]]

## Word2Vec

2013年，Google开源了一款直接计算低微词向量的工具----Word2Vec，不仅能够在百万级的词典亿级数据集上高效训练，而且能够很好的度量词与词之间的相似性。

对原始NNLM的改进：
移除前向反馈神经网络中的非线性hidden layer，直接将中间层的embedding layer 与 softmax layer连接
输入所有词向量到一个embedding layer 中
将特征词嵌入上下文环境
后续还在训练方法上进行了优化：层次softmax以及负采样技术

word2vec两种训练方式：
Continuous Bag of Words Model和Skip-Gram
word2vec两种优化方式：
hierarchical softmax 和negative sampling

模型步骤见资料https://blog.csdn.net/mawenqi0729/article/details/80698780

In [ ]:
from gensim.models import Word2Vec
model = Word2Vec(sentences=None, # 被训练的文本数据
                 corpus_file=None, # 语料的路径，可以替换掉sentence参数
                 size=100, # 词向量的维度
                 alpha=0.025, # 初始学习率
                 window=5, # 滑动窗口，左右各5个词
                 min_count=5, # 最小计数，如果单词统计数不够就不会进行训练
                 max_vocab_size=None, # 构建词汇表时的内存限制，默认不限制
                 sample=1e-3, # 对高频词进行下采样，采样比例阈值设置
                 seed=1, # 随机种子，初始化词向量时使用，单词的哈希值+seed
                 workers=3, # 线程数
                 min_alpha=0.0001, # 学习率会随迭代次数线性衰减
                 sg=0, # 1表示是skip-gram，0表示CBOW
                 hs=0, # 1表示层级softmax，0表示负采样
                 negative=5, # 如果>0表示使用负采样，官方实验推荐5-20
                 ns_exponent=0.75, # 负采样指数分布，官方推荐0.75
                 cbow_mean=1, # CBOW算法词向量的的合并，0表示使用词向量的和，1表示使用词向量的均值
                 hashfxn=hash, # 哈希的方式初始化权重，有利于复现权重
                 iter=5, # 迭代次数
                 null_word=0, 
                 trim_rule=None, 
                 sorted_vocab=1, # 1表示按照单词出现频率的降序进行排序
                 batch_words=MAX_WORDS_IN_BATCH, # 批次
                 compute_loss=False, # 是否计算并保持损失
                 callbacks=(),
                 max_final_vocab=None)
                 # 保存模型
model.sava(fname)
# 保存词向量，以数值的方式保存，非二进制
model.wv.save_word2vec_format('../data/embedding.txt',binary=False)
# 加载
model = Word2Vec.load(fname)
# 模型的使用
model.most_similar(positive = ['woman','king'],negative = ['man'])
# 输出：[('queen',0.50882536),...]
model.doesnt_match("brekfast cereal dinner lunch".split())
# output:'cereal'
model.similarity('woman','man')
# output:0.737
model['computer']	# raw numpy vector of a word
# output:array([-0.00449447, -0.00310097, 0.02421786, ...],dtype = float32)


In [4]:
from gensim.models import Word2Vec

sentences= [["cat", "say", "meow"],["dog", "say", "woof"]]

model = Word2Vec(sentences, min_count=1)
 
# sentences=word2vec.Text8Corpus(r'D:\\file_name)#训练集的格式为英文文本或分好词的中文文本

print(model)
#计算词向量的相似度
sim1 = model.similarity('cat', 'dog')
print(sim1)

Word2Vec(vocab=5, size=100, alpha=0.025)
-0.010127383392667244


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  # This is added back by InteractiveShellApp.init_path()


In [8]:
from gensim.models import Word2Vec

sentences= [["cat", "say", "meow"],["dog", "say", "woof"]]
model_1 = Word2Vec(min_count=1)
model_1.build_vocab(sentences)  # prepare the model vocabulary
model_1.train(sentences, total_examples=model.corpus_count, epochs=model.iter)  # train word vectors

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  


(1, 30)